### connection with calendar
### look up 1 week from now get all the events with paddle
### delete all events with paddle
### parse data from appointments json
### create corresponding event

# Connection with calendar --> my_calendar
#appointments.json ne contient pas l'annee
il va faloir dire 2025 par defaut

In [1]:

# iCloud credentials
from dotenv import load_dotenv
import os
from datetime import datetime, date, timedelta
from caldav import DAVClient
import uuid
import os
import pytz
import icalendar

# iCloud credentials

CALDAV_USER = os.getenv("CALDAV_USER")
CALDAV_PASSWORD = os.getenv("CALDAV_PASSWORD")  # Generate at appleid.apple.com
CALDAV_URL = 'https://caldav.icloud.com'

In [ ]:
# iCloud credentials
from dotenv import load_dotenv
import os
from datetime import datetime, date, timedelta
from caldav import DAVClient
import uuid
import os
import pytz
import icalendar

# iCloud credentials

CALDAV_USER = os.getenv("CALDAV_USER")
CALDAV_PASSWORD = os.getenv("CALDAV_PASSWORD")  # Generate at appleid.apple.com
CALDAV_URL = 'https://caldav.icloud.com'

def get_calendar(caldav_url,caldav_username,caldav_password, calendar_name):
    # link calendar
    client = DAVClient(url=caldav_url,username=caldav_username, password=caldav_password)
    calendar = client.principal().calendar(name=calendar_name)
    return calendar
 

def find_pickleball_events(calendar,start_date = None, search_list = ['🏓', '🎾' ]):
    if start_date is None:
        start_date = datetime.now() +timedelta(days=1)
    end_date = start_date + timedelta(days=7)
    events = calendar.search(start=start_date, end=end_date)
    pickleball_events = []

    for event in events:
       cal = icalendar.Calendar.from_ical(event.data)  # Parse les bytes vers objet Calendar
       for component in cal.walk():
           if component.name == "VEVENT":
               summary = str(component.get('SUMMARY', ''))
           # Vérifier si l'emoji tennis est présent
               for search_character in search_list:
                   if search_character in summary:
                       pickleball_events.append({
                        'summary': summary,
                        'start': component.get('dtstart').dt,
                        'end': component.get('dtend').dt,
                        'uid': str(component.get('uid'))
                    })
    
    return pickleball_events


def delete_events(calendar,list_events):
    uids_to_delete = [event['uid'] for event in list_events]
    for uid in uids_to_delete:
        try:
            event = calendar.event_by_uid(uid)
            event.delete()
        except:
            print(f"failed to delete:{uid}")



def parsed_events_from_appointments_json(year=2025, remove_today= True):
    import json
    
    with open("appointments.json", 'r') as f:
        events_data = json.load(f)

    parsed_events = []
    for item in events_data:
    # Parse date (format: "Wed\n22\nOct")
        date_parts = item['date'].split('\n')
        day = int(date_parts[1])
        month_str = date_parts[2]
        
        # Convert month name to number
        months = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
                    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
        month = months[month_str]
        time_parts = item['time'].replace('\n',' ').split(' - ')
        start = datetime.strptime(f"{year}-{month:02d}-{day:02d} {time_parts[0]}", 
                                    "%Y-%m-%d %I:%M %p")

        end = datetime.strptime(f"{year}-{month:02d}-{day:02d} {time_parts[1]}", 
                                "%Y-%m-%d %I:%M %p")
        parsed_events.append({
                'start': start,
                'end': end,
                'summary': '🏓 registration Pickleball'
            })
    if remove_today:
        today = date.today()
        future_events = [e for e in parsed_events if e['start'].date() > today]
        return future_events
    else:
        return parsed_events




def create_events(calendar, list_events):
    from icalendar import Calendar, Event

    # Add each event
    for event_data in list_events:
        cal = Calendar()
        event = Event()
        
        event.add('summary', event_data['summary'])
        event.add('dtstart', event_data['start'])
        event.add('dtend', event_data['end'])
        event.add('uid', f"pickleball-{event_data['start'].strftime('%Y%m%d-%H%M')}")
        
        cal.add_component(event)
        calendar.save_event(cal.to_ical())
        
        print(f"Added: {event_data['start'].strftime('%b %d %I:%M %p')}")




def main():
    #get calendar
    calendar = get_calendar(CALDAV_URL, CALDAV_USER, CALDAV_PASSWORD, "Family")
    # clean up
    list_events_to_delete = find_pickleball_events(calendar)
    delete_events(calendar, list_events_to_delete)
    list_events_to_add = parsed_events_from_appointments_json()
    create_events(calendar,list_events_to_add)
    

In [21]:
main()

Added: Oct 24 02:45 PM
Added: Oct 25 08:30 AM
Added: Oct 27 02:30 PM


In [37]:
start = datetime.strptime(f"{year}-{month:02d}-{day:02d} {time_parts[0]}", 
                                 "%Y-%m-%d %I:%M %p")
start

end = datetime.strptime(f"{year}-{month:02d}-{day:02d} {time_parts[1]}", 
                               "%Y-%m-%d %I:%M %p")
end


datetime.datetime(2025, 10, 27, 16, 30)

In [ ]:
from datetime import datetime

def parse_events_json(json_file, year = 2025):
    """Parse JSON file and return list of events with datetime objects"""
    import json
    
    with open(json_file, 'r') as f:
        events_data = json.load(f)
    
    parsed_events = []
    
    for item in events_data:
        # Parse date (format: "Wed\n22\nOct")
        date_parts = item['date'].split('\n')
        day = int(date_parts[1])
        month_str = date_parts[2]
        
        # Convert month name to number
        months = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
                  'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
        month = months[month_str]
        year = 2025
        
        # Parse time (format: "12:30 PM -\n2:30 PM")
        time_parts = item['time'].replace('\n', '').split(' - ')
        
        # Create datetime objects
        start = datetime.strptime(f"{year}-{month:02d}-{day:02d} {time_parts[0]}", 
                                 "%Y-%m-%d %I:%M %p")
        end = datetime.strptime(f"{year}-{month:02d}-{day:02d} {time_parts[1]}", 
                               "%Y-%m-%d %I:%M %p")
        
        parsed_events.append({
            'start': start,
            'end': end,
            'summary': '🏓 registration Pickleball'
        })
    
    return parsed_events

# Usage
events = parse_events_json('appointments.json')
for event in events:
    print(f"{event['start'].strftime('%B %d, %Y %I:%M %p')} - {event['end'].strftime('%I:%M %p')}")

ValueError: unconverted data remains:  -2:30 PM

In [14]:
pickleball_events = find_pickleball_events(my_calendar)

uids_to_delete = [event['uid'] for event in pickleball_events]
uids_to_delete

[]

In [ ]:
def delete_events(calendar,list_events):
    uids_to_delete = [event['uid'] for event in list_events]
    for uid in uids_to_delete:
        try:
            event = calendar.event_by_uid(uid)
            event.delete()
        except:
            print(f"failed to delete:{uid}")




In [17]:
delete_events(my_calendar,pickleball_events)

In [7]:
#find event horizon 1 week with paddle

# Dates pour la semaine à venir
start_date = datetime.now()
end_date = start_date + timedelta(days=7)
events = my_calendar.search(start=start_date, end=end_date)
pingpong_events = []

for event in events:
   cal = icalendar.Calendar.from_ical(event.data)  # Parse les bytes vers objet Calendar
   for component in cal.walk():
      if component.name == "VEVENT":
         summary = str(component.get('SUMMARY', ''))
         # Vérifier si l'emoji tennis est présent
         if '🏓' or '🎾' in summary:
            print(f"Événement tennis trouvé: {summary}")
            pingpong_events.append({
                        'summary': summary,
                        'start': component.get('dtstart').dt,
                        'end': component.get('dtend').dt,
                        'uid': str(component.get('uid'))
                    })






Événement tennis trouvé: Pickleball Lessons Confirmation
Événement tennis trouvé: 🎾 Your Pickleball Appointments - 4 total
Événement tennis trouvé: 🎾 Your Pickleball Appointments - 4 total


In [8]:
pingpong_events

[{'summary': 'Pickleball Lessons Confirmation',
  'start': datetime.datetime(2025, 10, 7, 13, 45, tzinfo=zoneinfo.ZoneInfo(key='America/Edmonton')),
  'end': datetime.datetime(2025, 10, 7, 15, 15, tzinfo=zoneinfo.ZoneInfo(key='America/Edmonton')),
  'uid': '2E15338E-27A5-4B46-9FF8-A49920EBC916'},
 {'summary': '🎾 Your Pickleball Appointments - 4 total',
  'start': datetime.datetime(2025, 10, 24, 14, 45, tzinfo=zoneinfo.ZoneInfo(key='America/Edmonton')),
  'end': datetime.datetime(2025, 10, 24, 16, 15, tzinfo=zoneinfo.ZoneInfo(key='America/Edmonton')),
  'uid': '35D8AA5E-3A6D-4038-97B4-36FF81319077'},
 {'summary': '🎾 Your Pickleball Appointments - 4 total',
  'start': datetime.datetime(2025, 10, 25, 8, 30, tzinfo=zoneinfo.ZoneInfo(key='America/Edmonton')),
  'end': datetime.datetime(2025, 10, 25, 10, 30, tzinfo=zoneinfo.ZoneInfo(key='America/Edmonton')),
  'uid': 'ED9B347F-2800-493D-A33A-7E0C4A4CD358'}]

In [ ]:
a={
    "date": "Sat\n25\nOct",
    "time": "8:30 AM -\n10:30 AM"
  }
day_num = int(a["date"].split("\n")[1])
month_num=month_map[a["date"].split("\n")[2]]

In [55]:
from datetime import datetime


In [89]:
start = a["time"].split("-\n")[0].replace(' ','')
dt_object_start = datetime.strptime(start, '%I:%M%p')
hour_start = dt_object_start.hour
minute_start = dt_object_start.minute
end = a["time"].split("-\n")[1].replace(' ','')
dt_object_end = datetime.strptime(end, '%I:%M%p')
hour_end = dt_object_end.hour
minute_end = dt_object_end.minute

In [91]:
import pytz


In [92]:

# Set timezone and create event
mst = pytz.timezone('America/Denver')
start = mst.localize(datetime(2025,month_num,day_num,hour_start,minute_start))
end = mst.localize(datetime(2025,month_num,day_num,hour_end,minute_end))

In [93]:
start

datetime.datetime(2025, 10, 25, 8, 30, tzinfo=<DstTzInfo 'America/Denver' MDT-1 day, 18:00:00 DST>)

In [90]:
datetime(2025,month_num,day_num,hour_start,minute_start,0)

datetime.datetime(2025, 10, 25, 8, 30)

In [88]:
hour_start

8

In [70]:
debut

'8:30 PM'

In [82]:
debut='8:30 AM'
dt_object = datetime.strptime(debut, '%I:%M %p')
hour = dt_object.hour
minute = dt_object.minute
hour


8

In [ ]:

b = datetime.strptime(debut, '%- I:%M %p ')

ValueError: '-' is a bad directive in format '%- I:%M %p '

In [ ]:
debut = a["time"].split("-\n")
    end =   time_parts[2] +time_parts[3]
    debut_h = datetime.strptime(debut, '%I:%M%p').strftime('%H:%M')
    end_h = datetime.strptime(debut, '%I:%M%p').strftime('%H:%M')

In [47]:
day_num

25

  {
    "date": "Sat\n25\nOct",
    "time": "8:30 AM -\n10:30 AM"
  }--> datetime(2025,10,25,8,30) - datetime(2025,10,25,10,30) 


    "date": "Mon\n27\nOct",
    "time": "2:30 PM -\n4:30 PM"
   }--> datetime(2025,10,27,14,30) - datetime(2025,10,27,6,30) 

In [1]:
paddle = "🏓"
ball = "🏐"
print(paddle, ball)

🏓 🏐


In [1]:
!pip install caldav pytz

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from caldav import DAVClient
from datetime import datetime, timedelta
import uuid
import os
import pytz
import icalendar


In [ ]:
# iCloud credentials
from dotenv import load_dotenv


# iCloud credentials

CALDAV_USER = os.getenv("username")
CALDAV_PASSWORD = os.getenv("password_calendar")  # Generate at appleid.apple.com
CALDAV_URL = 'https://caldav.icloud.com'


In [7]:
client = DAVClient(url=CALDAV_URL,username=CALDAV_USER, password=CALDAV_PASSWORD)

In [ ]:
principal = client.principal()
for cal in principal.calendars():
    print(cal.name)
    print(cal.url)

In [9]:
my_calendar = principal.calendar(name="Family")


In [10]:
# Dates pour la semaine à venir
start_date = datetime.now()
end_date = start_date + timedelta(days=7)


In [11]:
events = my_calendar.search(start=start_date, end=end_date)
for event in events:
   cal = icalendar.Calendar.from_ical(event.data)  # Parse les bytes vers objet Calendar
   for component in cal.walk():
      if component.name == "VEVENT":
         summary = str(component.get('SUMMARY', ''))
         # Vérifier si l'emoji tennis est présent
         if '🎾' in summary:
            print(f"Événement tennis trouvé: {summary}")





Événement tennis trouvé: 🎾 Your Pickleball Appointments - 4 total
Événement tennis trouvé: 🎾 Your Pickleball Appointments - 4 total


In [12]:
# Charger le JSON
import json
with open('appointments.json', 'r') as f:
    events = json.load(f)


In [ ]:
events
year =2025


[{'date': 'Wed\n22\nOct', 'time': '12:30 PM -\n2:30 PM'},
 {'date': 'Fri\n24\nOct', 'time': '2:45 PM -\n4:15 PM'},
 {'date': 'Sat\n25\nOct', 'time': '8:30 AM -\n10:30 AM'},
 {'date': 'Mon\n27\nOct', 'time': '2:30 PM -\n4:30 PM'}]

In [30]:
# Parser et ajouter chaque événement
for event_data in events:
    # Nettoyer et parser la date
    date_parts = event_data['date'].replace('\n', ' ').split()  # ['Wed', '22', 'Oct']
    day_num = date_parts[1]
    month = date_parts[2]
    time_parts = event_data['time'].replace('-\n', ' ').split()  # ['Wed', '22', 'Oct']
    debut = time_parts[0] + time_parts[1]
    end =   time_parts[2] +time_parts[3]
    debut_h = datetime.strptime(debut, '%I:%M%p').strftime('%H:%M')
    end_h = datetime.strptime(debut, '%I:%M%p').strftime('%H:%M')
    year=2025
    print(year)
    print(month)
    print(day_num)
    print(end_h)





2025
Oct
22
12:30
2025
Oct
24
14:45
2025
Oct
25
08:30
2025
Oct
27
14:30


In [ ]:
    "date": "Sat\n25\nOct",


In [37]:
date_string = str(day_num)+"/"+month+"/"+str(year)

date_string

'27/Oct/2025'

In [39]:
dt_object

datetime.datetime(2025, 10, 27, 0, 0)

In [38]:
from datetime import datetime

dt_object = datetime.strptime(date_string, "%d/%b/%Y")
month_number = dt_object.month
print(f"The number for October in {date_string} is: {month_number}")

The number for October in 27/Oct/2025 is: 10


In [29]:
debut
dt_object = datetime.strptime(debut, '%I:%M%p')

time_24hr_str = dt_object.strftime('%H:%M')
time_24hr_str


'14:30'

In [20]:
time_parts

['2:30', 'PM', '4:30', 'PM']

In [14]:

# Parser et ajouter chaque événement
for event_data in events:
    # Nettoyer et parser la date
    date_parts = event_data['date'].replace('\n', ' ').split()  # ['Wed', '22', 'Oct']
    day_num = date_parts[1]
    month = date_parts[2]
    
    # Nettoyer et parser les heures
    time_clean = event_data['time'].replace('\n', '')  # "12:30 PM -2:30 PM"
    start_time, end_time = time_clean.split(' - ')
    
    # Construire les datetime (année 2024)
    year = 2025
    start = datetime.strptime(f"{day_num} {month} {year} {start_time}", "%d %b %Y %I:%M %p")
    end = datetime.strptime(f"{day_num} {month} {year} {end_time}", "%d %b %Y %I:%M %p")
    


ValueError: not enough values to unpack (expected 2, got 1)